In [1]:
import tdmclient.notebook
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

In [2]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    obst = prox_horizontal
    return obst

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""


TRESH_DIST = 10 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436*1.1
YAW_TO_DEGS = 0.06086*4
SPEED_OFFSET = 5
FROM_KHALMAN = 0
FROM_CAMERA = 1
GOAL = 2


def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # We need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("Take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
            
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        # wtf il faut virer ça
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
                
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        optimal_trajectory = goal_list.copy()
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1] 
        # [mm, mm, degrees] 
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        # [mm/s, degrees/s] 
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[1,  0,   0],
                                [0,  1,   0],
                                [0,  0,   1]])
    
        dt = 0.1 
        v_l = 0
        v_r = 0
        state = 0
        obstThrH = 1400
        
        time.sleep(0.1)
        
        while True:

            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
            # Returns the camera measurement of the robot position and a boolean 
            # indicating if the camera is hidden or not
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, FROM_CAMERA)
            
            computer_vision.display_path(frame, optimal_trajectory, px_to_mm)
              
            # Run the Kalman Filter and store the 
            # optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.kf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Indicator of the camera state
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, FROM_KHALMAN)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, GOAL)
            
            frame = cv2.resize(frame, None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('Running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                if len(goal_list) == 0:
                    break
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            # Check if obstacle in coming
            obst = get_prox()
            
            # Put a name to the state instead of a number? state = "local avoidance" or "global path"
            if state == 0:
                for i in range(len(obst)-2):
                # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > obstThrH):
                        print('obst', obst)
                        state = 1
        
            if state == 0:
                # Call P controler to obtain the speed of the robot
                v_l,v_r = filters.p_controler(optimal_state_estimate_t,goal_list[0])
            
            else:
            # If we are in local avoidance state
                v, state = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r, True)
                v_l = v[0]
                v_r = v[1]
            
            # Conversion of speed and yaw to real world values
            v = (v_l + v_r)*SPEED_TO_MMS/2
            yaw = (v_l-v_r)*YAW_TO_DEGS
            u_t_minus_1 = [v, yaw]; 
            
            # Setting the speed of the robot, correct the fact that the robot doesn't naturally go straight 
            set_speed(int(v_l), int(v_r+SPEED_OFFSET))

            actual = time.time()
            diff = actual-previous
            if diff < dt:
                time.sleep(dt-diff)
            
    finally:
        set_speed(0, 0)
        cv2.waitKey(0)
        cv2.destroyAllWindows()



In [17]:
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [6]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [7]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [19]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

<module 'obstacle_avoidance' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\obstacle_avoidance.py'>

In [ ]:
main()

Take the rigth image
Px_to_mm=  0.4816497514266023


100%|██████████| 1/1 [00:00<00:00, 67.58it/s]


In [ ]:
cap.release()

In [16]:
await tdmclient.notebook.stop()